In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
import json
from datasets import Dataset


In [2]:
def read_json_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return pd.DataFrame(data) 

data = read_json_from_file("./data/refined_hindi_data.json")
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['instruction', 'output'],
    num_rows: 14936
})

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")

def tokenize_function(examples):
    inputs = tokenizer(examples["instruction"], examples["output"], truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["instruction", "output"])



Map:   0%|          | 0/14936 [00:00<?, ? examples/s]

In [4]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B", device_map="auto")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("Starting training...")
trainer.train()

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Starting training...


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.16 GiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Process 10542 has 17179869184.00 GiB memory in use. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 13.16 GiB is allocated by PyTorch, and 671.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)